# Correlation Analysis


In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import seaborn as sns
import plotly.graph_objects as go




from fault_management_uds.data.load import import_external_metadata, import_metadata
from fault_management_uds.data.HDF5_functions import print_tree, load_dataframe_from_HDF5
from fault_management_uds.plots import visualize_indicator_dict



from fault_management_uds.config import PROJ_ROOT
from fault_management_uds.config import DATA_DIR, RAW_DATA_DIR, INTERIM_DATA_DIR, PROCESSED_DATA_DIR, EXTERNAL_DATA_DIR
from fault_management_uds.config import MODELS_DIR, REPORTS_DIR, FIGURES_DIR, REFERENCE_DIR
from fault_management_uds.config import natural_sensor_order, natural_structure_order, structure_2_sensor, error_indicators, bools_2_meta, rain_gauges

2024-11-15 15:46:19.147 | INFO     | fault_management_uds.config:<module>:11 - PROJ_ROOT path is: /Users/arond.jacobsen/Documents/GitHub/fault_management_uds


In [3]:
data_file_path = PROCESSED_DATA_DIR / 'Bellinge.h5'
f = print_tree(data_file_path)

root
├── combined_data
│   ├── clean
│   │   ├── columns
│   │   ├── data
│   │   └── timestamps
│   └── raw
│       ├── columns
│       ├── data
│       └── timestamps
└── single_series
    ├── rain_gauge_data
    │   ├── 5425
    │   │   ├── columns
    │   │   ├── data
    │   │   └── timestamps
    │   └── 5427
    │       ├── columns
    │       ├── data
    │       └── timestamps
    └── sewer_data
        ├── G71F04R_Level1
        │   ├── bools
        │   │   ├── columns
        │   │   ├── data
        │   │   └── timestamps
        │   ├── clean
        │   │   ├── columns
        │   │   ├── data
        │   │   └── timestamps
        │   └── raw
        │       ├── columns
        │       ├── data
        │       └── timestamps
        ├── G71F04R_Level2
        │   ├── bools
        │   │   ├── columns
        │   │   ├── data
        │   │   └── timestamps
        │   ├── clean
        │   │   ├── columns
        │   │   ├── data
        │   │   └── timestamps
        │ 

In [4]:
sensor_metadata = import_metadata(REFERENCE_DIR / 'sensor_metadata.csv')
external_metadata = import_metadata(REFERENCE_DIR / 'external_metadata.csv')
starttime = sensor_metadata['StartTime'].min()
endtime = sensor_metadata['EndTime'].max()

In [5]:
save_folder = FIGURES_DIR / 'analysis' / 'correlation'
os.makedirs(save_folder, exist_ok=True)

### Load clean data

In [6]:
clean, _, _, _ = load_dataframe_from_HDF5(data_file_path, 'combined_data/clean')

In [7]:
# restructure columns
clean = clean[rain_gauges + natural_sensor_order]

### Load rain events

In [30]:
# load the rain events
rain_events = pd.read_csv(REFERENCE_DIR / 'clf_rain_events.csv')
rain_events['start'] = pd.to_datetime(rain_events['start'])
rain_events['end'] = pd.to_datetime(rain_events['end'])
print(rain_events.shape)
rain_events.head()

(2033, 4)


,start,end,duration,total_rain
0,2010-01-11 18:37:00,2010-01-12 03:12:00,530,123.296
1,2010-01-27 13:53:00,2010-01-27 16:17:00,159,36.689
2,2010-01-29 23:51:00,2010-01-30 02:21:00,165,43.327
3,2010-01-31 02:53:00,2010-01-31 03:48:00,70,23.345
4,2010-02-02 11:22:00,2010-02-02 15:20:00,253,69.968


### Filter data

In [32]:
# filter data to only include rain events
# - want 3 hours before and 3 hours after
timestamps = []
for _, row in rain_events.iterrows():
    timestamps.extend(pd.date_range(row['start'] - pd.Timedelta(hours=3), row['end'] + pd.Timedelta(hours=3), freq='1min'))

filtered_clean = clean.loc[timestamps].copy()

## Matrix correlation

In [38]:
corr = clean.corr()


# Create the mask
mask = np.triu(np.ones_like(corr, dtype=bool))

# Apply the mask to the correlation matrix
masked_corr = corr.mask(mask)


In [39]:

# Create the heatmap trace
heatmap = go.Heatmap(
    z=corr.values,
    x=corr.columns.tolist(),
    y=corr.index.tolist(),
    # color from blue to red
    colorscale='RdBu',
    #colorscale='RdYlGn', 
    # add text in each cell
    text=corr.values,
    texttemplate='%{text:.2f}',
    hovertemplate='Correlation: %{text:.2f}<extra></extra><br>%{y}<br>%{x}',
    showscale=True,
    reversescale=False,
    zmin=-1,
    zmax=1,
    xgap=1,
    ygap=1,
)


# Create the layout
layout = go.Layout(
    #title='Correlation Heatmap',
    xaxis=dict(tickangle=-45),  # Rotate x-axis labels
    yaxis=dict(autorange='reversed'), # Invert the y-axis
    height=900,
    width=1100,
)

# Create the figure
fig = go.Figure(data=[heatmap], layout=layout)
# Save the plot as png
fig.write_image(save_folder / 'correlation_matrix.png', scale=2)
# tight layout
fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
# Show the plot
fig.show()


- Some level sensors correlate completely, but they measure the same thing
    - i.e. one increases and the other increases proportionally
- Position is negative, because typically locked gate/low during heavy rain, e.g. especially its basin
- Rain gauges don't have so much correlation
    - maybe band would be interesting to investigate?

- does it make sense to model base don close/clustered sensor

In [35]:
corr = filtered_clean.corr()


# Create the mask
mask = np.triu(np.ones_like(corr, dtype=bool))

# Apply the mask to the correlation matrix
masked_corr = corr.mask(mask)


In [37]:

# Create the heatmap trace
heatmap = go.Heatmap(
    z=corr.values,
    x=corr.columns.tolist(),
    y=corr.index.tolist(),
    # color from blue to red
    colorscale='RdBu',
    #colorscale='RdYlGn', 
    # add text in each cell
    text=corr.values,
    texttemplate='%{text:.2f}',
    hovertemplate='Correlation: %{text:.2f}<extra></extra><br>%{y}<br>%{x}',
    showscale=True,
    reversescale=False,
    zmin=-1,
    zmax=1,
    xgap=1,
    ygap=1,
)


# Create the layout
layout = go.Layout(
    #title='Correlation Heatmap',
    xaxis=dict(tickangle=-45),  # Rotate x-axis labels
    yaxis=dict(autorange='reversed'), # Invert the y-axis
    height=900,
    width=1100,
)

# Create the figure
fig = go.Figure(data=[heatmap], layout=layout)
# Save the plot as png
fig.write_image(save_folder / 'correlation_matrix_filtered.png', scale=2)
# tight layout
fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
# Show the plot
fig.show()


## Cross-Correlation

If a variable is constant, it flattens the line, since no variation is there to correlate with
- thus, we remove values, if the base variable continues to be zero beyond the lags we are checking

In [54]:
fig = plot_cross_correlation(filtered_clean, '5425', lag_range=120, width=1100, height=450) # width=900, height=450):
fig.write_image(save_folder / 'cross_correlation.png', scale=2)
fig.show()



In [56]:
fig = plot_cross_correlation(filtered_clean, '5427', lag_range=60, width=1100, height=450) # width=900, height=450):
fig.show()



Interpretation:

- Peak at Lag 3: This indicates that the second time series is significantly correlated with the first time series when shifted three time units (e.g., days, weeks, months) ahead.
    - pos: second leads the first
    - neg: first leads the second
- Negative Peak: A negative peak suggests an inverse relationship. When one series increases, the other decreases.


In [ ]:
rain_gauges

['5425', '5427']

In [ ]:
base_variable = '5425'
plot_cross_correlation(raw, base_variable, lag_range=120)

- So 5425 precedes 5427 but the other varialbes do not precede 5427

#### Cross-Correlatio: Decending down-stream

In [ ]:
# for i, structure in enumerate(natural_structure_order):
#     print(structure)
#     columns = list(raw.columns)
#     structure_vars = [col for col in columns if structure in col]
#     structure_idxs = [columns.index(var) for var in structure_vars]
#     min_index = min(structure_idxs)
#     for i, structure_var in enumerate(structure_vars):
#         print(structure_var)
#         plot_cross_correlation(raw.iloc[:, min_index:], structure_var, lag_range=120, width=900, height=300)


- Nothing interesting

#### Cross-Correlation: circular relationships

In [ ]:
# circular_relationship = ['G80F11B', 'G80F66Y']
# cricular_relationship_vars = [col for col in columns if any([structure in col for structure in circular_relationship])]
# for base_variable in cricular_relationship_vars:
#     print(base_variable)
#     plot_cross_correlation(raw[cricular_relationship_vars], base_variable, lag_range=120)


- No interesting relation, just similar, and relation weakens for higher lags.
- All have a soft normal distribution shape

In [ ]:
circular_relationship = ['G71F06R', 'G71F68Y']
cricular_relationship_vars = [col for col in columns if any([structure in col for structure in circular_relationship])]
for base_variable in cricular_relationship_vars:
    print(base_variable)
    plot_cross_correlation(raw[cricular_relationship_vars], base_variable, lag_range=120)

G71F06R_LevelInlet


G71F68Y_LevelPS


G71F68Yp1


- Increased flow from pumping -> increased level, how come?

#### Cross-Correlation with storage pipe

In [ ]:
storage_pipe_relation = ['G71F04R', 'G71F05R', 'G71F06R', 'G71F68Y']
storage_pipe_relation_vars = [col for col in columns if any([structure in col for structure in storage_pipe_relation])]
storage_pipe_relation_vars += ['5425', '5427']
storage_pipe_vars = [col for col in columns if 'G71F68Y' in col]

for base_variable in storage_pipe_vars:
    print(base_variable)
    plot_cross_correlation(raw[storage_pipe_relation_vars], base_variable, lag_range=120)

G71F68Y_LevelPS


G71F68Yp1


- ?

### TODO:
- auto correlation as well?
